## Jupyter Notebook

This notebook created for Data Sciense Capstone Project Course.
Part of IBM Data Science Professional Certificate.

Tutor: Alex Aklson                                          
Student: Iurii Karavan
https://www.linkedin.com/in/yury-karavan/

April 2020, Saint-Petersburg, Russia

### Import Section

Let's install modules, if needed. Installation and tweaking of your basic python is out of this notebook.

check these modules are installed
numpy \
pandas\
lxml\
html5lib and bs4\
geocoder (if you have a billing Google account)\
folium

In [1]:
import pandas as pd
import numpy as np
#import geocoder
import folium 
print('Modules are loaded')

Modules are loaded


### Code section

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


Preparing variables for DataFarme

In [3]:
# original link is 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# in accordance to pandas.read_html docs will change https protocol to http
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
header = 0
flavor = 'html5lib'
match = 'Postal code'
skiprows = [0,1]
df = pd.DataFrame()

In [4]:
# full command is pandas.read_html(io, match='.+', flavor=None, header=None, index_col=None, skiprows=None, attrs=None, parse_dates=False, tupleize_cols=None, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
dfs = pd.read_html(url, match=match, flavor=flavor, skiprows=skiprows)

In [5]:
#Let's merge list od DataFrames to the singl DataFrama
df = pd.concat(dfs)

In [6]:
#Give names to columns of DataFrame
df.columns=['PostalCode', 'Borough', 'Neighborhood']

In [7]:
#check size of DataFrame
df.shape

(179, 3)

In [8]:
#Let's see first 10 rows
df.head(10)

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights
5,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
6,M8A,Not assigned,NaN
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Malvern / Rouge
9,M2B,Not assigned,NaN


In [9]:
#Let's see last 5 rows
df.tail()

,PostalCode,Borough,Neighborhood
174,M5Z,Not assigned,NaN
175,M6Z,Not assigned,NaN
176,M7Z,Not assigned,NaN
177,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
178,M9Z,Not assigned,NaN


Let's remove rows where 'Borough' is 'Not assigned', reset indexes and change "/" separator to comma

In [10]:
for i in range(len(df)):
    if df.loc[i, 'Borough'] == 'Not assigned':
        df.drop(i, inplace=True)

df.reset_index(drop = True, inplace=True)

for i in range(len(df)):
    if ' / ' in df.loc[i, 'Neighborhood']:
        df.loc[i, 'Neighborhood'] = df.loc[i, 'Neighborhood'].replace(r' / ', ', ')
        
#check the 'Neighborhood' field  for 'Not assigned' values
if 'Not assigned' in df['Neighborhood']:
    print('Neighborhood field must be corrected')
else:
    print('Neighborhood field OK')    

Neighborhood field OK


In [11]:
#See first rows
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
#Check last rows
df.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [13]:
#check new size of our DataFrame
df.shape

(103, 3)

### Adding coordinates to DataFrame
Add columns to Data Frame

In [14]:
df['Latitude'] = 0.0
df['Longitude'] = 0.0

In [15]:
#show out names of columnns
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

## Geocoder
Using of geocoder is payable (5 USD for 1000 requests). In this reason I will use coordinates from the kindly provided by Alex Aklson file.\
about using Geocoder please read:\
https://developers.google.com/maps/documentation/geocoding/start?hl=ru&utm_source=google&utm_medium=cpc&utm_campaign=FY18-Q2-global-demandgen-paidsearchonnetworkhouseads-cs-maps_contactsal_saf&utm_content=text-ad-none-none-DEV_c-CRE_375946496075-ADGP_Hybrid+%7C+AW+SEM+%7C+SKWS+~+Places+%7C+EXA+%7C+Geocoding+API-KWID_43700046387144977-kwd-301485311002-userloc_9047064&utm_term=KW_geocoding%20api-ST_geocoding+api&gclid=CjwKCAjwg6b0BRBMEiwANd1_SH3iqXMXEVvxK6vc1LfRO8tcAG1Pw9hPWH7F3J6Xz41QOf39yzMZAhoCW4wQAvD_BwE

In [16]:
#let's data with original name
#I think downloading file is better than reading direct from a site because we need to do it once. 
#Next time we will use csv file.
#Do not forget to add this file in git or your CVS repository!

#!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

print('data loaded')

data loaded


In [17]:
#read data from file and change indexes to "Postal Code"
dfcoord = pd.read_csv('Geospatial_Coordinates.csv')
dfcoord.index = dfcoord['Postal Code']
dfcoord.head(3)

,Postal Code,Latitude,Longitude
Postal Code,,,
M1B,M1B,43.806686,-79.194353
M1C,M1C,43.784535,-79.160497
M1E,M1E,43.763573,-79.188711


In [18]:
#compare shapes od our dataframes. If lenghts are not equal we must stop processing
if len(df) == len (dfcoord):
    print('The lenghts of the DataFrames are equal')
else:
    print('You need to check your data!!')
    raise SystemExit(3)

The lenghts of the DataFrames are equal


Let's add coordinates to the main DataFrame and check data

In [19]:
for i in range(len(df)):
    p_code = df.loc[i, 'PostalCode']
    df.loc[i, ['Latitude','Longitude']] = dfcoord.loc[p_code, ['Latitude','Longitude']]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
